## 6. Elliptic Curve Signatures: ECDSA or EdDSA
### Alicja Lis 151569
### Mikołaj Lewandowski

6. Elliptic Curve Signatures: ECDSA or EdDSA
Goal: Implement ECDSA or EdDSA (e.g., using Curve25519 or P-256).
Subtasks:
* Easy: Use a cryptographic library (e.g., Python's cryptography, or libsodium) to generate and verify signatures.
* Medium: Implement signature and verification from scratch (using provided curve parameters).
* Hard: Simulate fault injection (e.g., nonce reuse) and show signature vulnerability.
* Bonus: Implement batch verification of multiple signatures.

### Easy: Use a cryptographic library (e.g., Python's cryptography, or libsodium) to generate and verify signatures.

In [5]:
from cryptography.hazmat.primitives import hashes

from cryptography.hazmat.primitives.asymmetric import ec
import hashlib
import secrets

private_key = ec.generate_private_key(

    ec.SECP256R1()

)
data = b"this is some data I'd like to sign"
tampered_data = b"this is tampered data"

signature = private_key.sign(

    data,

    ec.ECDSA(hashes.SHA256())

)

public_key = private_key.public_key()

print("Test for normal data:")
try:
    public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))
    print("Signature is valid")
except:
    print("Signature is not valid")

print("Test fot tampered data:")
try:
    public_key.verify(signature, tampered_data, ec.ECDSA(hashes.SHA256()))
    print("Signature is valid")
except:
    print("Signature is not valid")




Test for normal data:
Signature is valid
Test fot tampered data:
Signature is not valid


### Medium: Implement signature and verification from scratch (using provided curve parameters).

In utils.py we implemented all arithmetic needed to perform ecdsa, we also defined there our parameters for the curve from NIST page.

In [6]:
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff # big prime so that there are no cycles
a = 0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc # coeffs
b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b # coeffs
G = (0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5) # starting point
n = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551 # order
h = 0x1

def get_p_256_params():
    return p, a, b, G, n
def encrypt_message(message):
    sha_signature = \
        hashlib.sha256(message.encode()).hexdigest()
    return sha_signature

def modinv(a,p):
    # Extended Euclidean Algorithm
    return pow(a, -1, p)  #a**(-1) mod p


class EllipticCurve:
    def __init__(self, p, a, b, G, n):
        self.p = p
        self.a = a
        self.b = b
        self.G = G
        self.n = n

    def add(self, P, Q):
        if P == (None, None):
            return Q
        if Q == (None, None):
            return P

        x1, y1 = P
        x2, y2 = Q
        if x1 == x2 and y1 != y2:
            return (None, None)

        if P != Q:
            # λ = (y2 - y1) / (x2 - x1) mod p
            lam = ((y2 - y1) * modinv(x2 - x1, self.p)) % self.p
        else:
            # λ = (3x1² + a) / (2y1) mod p
            lam = ((3 * x1 * x1 + self.a) * modinv(2 * y1, self.p)) % self.p

        x3 = (lam * lam - x1 - x2) % self.p
        y3 = (lam * (x1 - x3) - y1) % self.p

        return(x3, y3)

    def multiply(self, k, P):
        result = (None, None)
        addend = P
        while k:
            if k & 1:
                result = self.add(result, addend)
            addend = self.add(addend, addend)
            k >>= 1
        return result


def generate_signature(message, private_key, curve, k=None):
    if k is None:
        k = secrets.randbelow(curve.n - 1) + 1

    z = int.from_bytes(hashlib.sha256(message.encode()).digest(), byteorder="big")

    # R = k * G
    R = curve.multiply(k, curve.G)
    r = R[0] % curve.n
    if r == 0:
        r = R[0] % curve.n

    # s = k⁻¹ (z + r * d) mod n
    s = (modinv(k, curve.n) * (z + r * private_key)) % curve.n
    if s == 0:
        s = (modinv(k, curve.n) * (z + r * private_key)) % curve.n

    return r, s

def verify_signature(message, r, s, public_key, curve):

    if r <= 0 or r >= curve.n or s <= 0 or s >= curve.n:
        return False

    z = int.from_bytes(hashlib.sha256(message.encode()).digest(), byteorder="big")
    w = modinv(s, curve.n)
    u1 = (z * w) % curve.n
    u2 = (r * w) % curve.n

    P = curve.add(curve.multiply(u1, curve.G), curve.multiply(u2, public_key))

    return P[0] % curve.n == r

Then in another file we just used the functions from utils.py

In [7]:
p, a, b, G, n = get_p_256_params()
curve = EllipticCurve(p, a, b, G, n)

private_key = secrets.randbelow(n - 1) + 1
public_key = curve.multiply(private_key, G)

# creating the signature
message = "ecdsa testing"
r, s = generate_signature(message, private_key, curve)

# verifying the signature
is_valid = verify_signature(message, r, s, public_key, curve)
if is_valid:
    print("signature is valid")
else:
    print("signature is invalid")


signature is valid


### Hard: Simulate fault injection (e.g., nonce reuse) and show signature vulnerability.

In order to simulate fault injection we generated two signatures for different messages using the same random nonce k. Because of this, we have the same r values for both signatures. After that we knew everything needed for the attack, applied simple system of equations and recovered private key d.

In [8]:
p, a, b, G, n = get_p_256_params()
k = secrets.randbelow(n - 1) + 1
d = secrets.randbelow(n - 1) + 1
curve = EllipticCurve(p, a, b, G, n)

message1 = "first message"
r1, s1 = generate_signature(message1, d, curve, k)

message2 = "second message"
r2, s2 = generate_signature(message2, d, curve, k)

if r1 == r2:
    print("r1 and r2 are the same")
else:
    print("r1 and r2 are different")

# Hash the messages
z1 = int(hashlib.sha256(message1.encode()).hexdigest(), 16)
z2 = int(hashlib.sha256(message2.encode()).hexdigest(), 16)

# simplified system of equations
k_recovered = ((z1 - z2) * modinv(s1 - s2, n)) % n

r_inv = modinv(r1, n)
d_recovered = ((s1 * k_recovered - z1) * r_inv) % n

print(f"Original d:  {d}")
print(f"Recovered d: {d_recovered}")

if d == d_recovered:
    print("Private key recovered due to nonce reuse")
else:
    print("keys don't match.")



r1 and r2 are the same
Original d:  65371152979888975435526665895986996511065174287965291599975455785020746454644
Recovered d: 65371152979888975435526665895986996511065174287965291599975455785020746454644
Private key recovered due to nonce reuse
